# Plota mapa diário de relâmpagos do GLM do satelite GOES-19
 - Dados GLM GOES-19 fornecido pelo CPTEC: http://ftp.cptec.inpe.br/goes/goes19/goes19_web/glm_acumulado_nc/
 - Realizado por: Enrique V. Mattos - 28/08/2025

# **1° Passo:** Preparando ambiente

In [ ]:
# instalações
!pip install -q ultraplot cartopy pygrib rioxarray salem rasterio pyproj geopandas

# monta o drive
from google.colab import drive
drive.mount('/content/drive')

# nome do diretório
dir = '/content/drive/MyDrive/2-PESQUISA/0_GLM/estudos_de_caso/2025-08-22-FRENTEFRIA_RS'

# importa bibliotecas
import ultraplot as uplt
import xarray as xr
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import os
import glob
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import time
from matplotlib import cm
import matplotlib.patches as patches
import pygrib
import matplotlib
import rioxarray as rxr
from datetime import datetime, timedelta
import pandas as pd
import salem

import warnings
warnings.filterwarnings("ignore")

# **2° Passo:** Declarando funções

In [ ]:
def evm_plot_states(shapefile, cor, espessura_linha):
    import cartopy.crs as ccrs
    import cartopy.io.shapereader as shpreader
    import matplotlib.pyplot as plt
    shapefile = list(shpreader.Reader(shapefile).geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor=cor, facecolor='none', linewidth=espessura_linha)

In [ ]:
# **Função que baixa os dados do GLM que possuem frequência temporal de 5 min no ftp do CPTEC**
def download_GLM5min_CPTEC(ano, mes, dia, hor, min):

    print('.... Baixando dado ===>', 'DATA=', ano, mes, dia, hor, min)

    # caminho do ftp do cptec
    ftp_cptec_glm_5min = 'http://ftp.cptec.inpe.br/goes/goes19/goes19_web/glm_acumulado_nc'

    # código do produto
    codigo_produto = 'S11162307'

    # nome do arquivo
    file_glm5min_cptec =  f'{ftp_cptec_glm_5min}/{ano}/{mes}/{codigo_produto}_{ano}{mes}{dia}{hor}{min}.nc' # S11162307_202507010000.nc

    # baixando a imagem de satélite
    !wget -c {file_glm5min_cptec}

    # local da imagem que foi baixada
    file_glm_download = f'/content/{codigo_produto}_{ano}{mes}{dia}{hor}{min}.nc'

    return file_glm_download

# **Processando os dados**

## Baixando os dados
Os dados serão baixados do ftp do CPTEC-INPE. São arquivos netcdf a cada 5 min. O tempo total para o download de 5 dias de dados é de aproximadamente 27 min.

In [ ]:
%%time
# ano, mes e dia INICIAL do período
anoi = 2025
mesi = 8
diai = 20

# ano, mes e dia FINAL do período
anof = 2025
mesf = 8
diaf = 24 # 24

# Datas no formato string
date_in = datetime(anoi, mesi, diai)
date_ini = date_in.strftime('%Y%m%d')

date_en = datetime(anof, mesf, diaf)
date_end = date_en.strftime('%Y%m%d')

# Download dos dados
files_imagens = []
for file in pd.date_range(date_ini, date_end, freq='5min'):

    # extrai ano e mes
    ano = file.strftime('%Y')
    mes = file.strftime('%m')
    dia = file.strftime('%d')
    dia = file.strftime('%d')
    hor = file.strftime('%H')
    min = file.strftime('%M')

    # baixa imagem de satélite
    file_glm_download = download_GLM5min_CPTEC(ano, mes, dia, hor, min)

    # salva o nomes das imagens baixadas
    files_imagens.append(file_glm_download)

## Processando os dados
A leitura dos dados demora aproximadamente 4 min.

In [ ]:
%%time
#==================================================================================================#
#                                   DEFINIÇÃO DOS LIMITES DA IMAGEM
#==================================================================================================#
# limites das latitudes e longitudes
lonmin, lonmax, latmin, latmax = -60.0, -40., -40., -20.0

# extensão da imagem [min. lon, min. lat, max. lon, max. lat]
extent = [lonmin, latmin, lonmax, latmax]

#==================================================================================================#
#                                     SHAPEFILES
#==================================================================================================#
shp_ES = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/ES/ES_UF_2019.shp').geometries())
shp_RJ = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/RJ/RJ_UF_2019.shp').geometries())
shp_SP = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/SP/SP_UF_2019.shp').geometries())
shp_PR = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/PR/PR_UF_2019.shp').geometries())
shp_SC = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/SC/SC_UF_2019.shp').geometries())
shp_RS = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/UFs/RS/RS_UF_2019.shp').geometries())

#==================================================================================================#
#                                   LEITURA DOS DADOS DO GLM-CPTEC
#==================================================================================================#
# DIA 20
flash_goes19_5min_dia20 = xr.open_mfdataset(glob.glob('/content/S11162307_20250820*nc'), concat_dim='time', combine='nested', decode_times=False).sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))

# DIA 21
flash_goes19_5min_dia21 = xr.open_mfdataset(glob.glob('/content/S11162307_20250821*nc'), concat_dim='time', combine='nested', decode_times=False).sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))

# DIA 22
flash_goes19_5min_dia22 = xr.open_mfdataset(glob.glob('/content/S11162307_20250822*nc'), concat_dim='time', combine='nested', decode_times=False).sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))

# DIA 23
flash_goes19_5min_dia23 = xr.open_mfdataset(glob.glob('/content/S11162307_20250823*nc'), concat_dim='time', combine='nested', decode_times=False).sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))

# DIA 24
flash_goes19_5min_dia24 = xr.open_mfdataset(glob.glob('/content/S11162307_20250824*nc'), concat_dim='time', combine='nested', decode_times=False).sel(lon=slice(lonmin, lonmax), lat=slice(latmin, latmax))

#==================================================================================================#
#                         SOMA OS RELÂMPAGOS PARA CADA DIA
#==================================================================================================#
flash_goes19_dia20 = flash_goes19_5min_dia20['flash'].sum(dim='time')
flash_goes19_dia21 = flash_goes19_5min_dia21['flash'].sum(dim='time')
flash_goes19_dia22 = flash_goes19_5min_dia22['flash'].sum(dim='time')
flash_goes19_dia23 = flash_goes19_5min_dia23['flash'].sum(dim='time')
flash_goes19_dia24 = flash_goes19_5min_dia24['flash'].sum(dim='time')

#==================================================================================================#
#                      SOMA O TOTAL DE RELÂMPAGOS ENTRE OS DIAS 20 e 24
#==================================================================================================#
flash_goes19_sum = flash_goes19_dia20 + flash_goes19_dia21 + flash_goes19_dia22 + flash_goes19_dia23 + flash_goes19_dia24

In [ ]:
# mostra os dados de 5min
flash_goes19_5min_dia20

In [ ]:
# mostra a matriz soma daquele dia
flash_goes19_dia20

In [ ]:
# matriz de soma
flash_goes19_sum

# **Plota figura**

In [ ]:
# mostrando o dado de FLASH que será utilizado
flash_goes19_dia20

In [ ]:
%%time
#==================================================================================================#
#                              DEFINIÇÕES DO GRÁFICO
#==================================================================================================#
# cria moldura da figura
fig, ax = uplt.subplots(ncols=3, nrows=2, axheight=6.9, axwidth=6.8, tight=True, proj='pcarree')

# formata os eixos
ax.format(coast=True, borders=True, innerborders=True,
          labels=True, latlines=4, lonlines=5,
          latlim=(latmin, latmax), lonlim=(lonmin, lonmax),
          abc=False, #abcstyle='(a)', abcsize=5
          small='40px', large='40px',
          linewidth=2,
          suptitle='Acumulado de Relâmpagos', suptitlecolor='red', suptitlesize=40)

# título de nome do produto
ax[3].text(lonmin+0.1, latmin-2.5, 'GOES-19 (GLM)/8km', color='grey', fontsize=30)

#==================================================================================================#
#                                       PLOTA FIGURAS
#==================================================================================================#
# datas das figuras
datas = ['2025-08-20', '2025-08-21', '2025-08-22', '2025-08-23', '2025-08-24','acumulado']

# loop das datas
for i, data in enumerate(datas):

    # define o dado de cada dia
    if data == '2025-08-20': flash_goes19_dia = flash_goes19_dia20
    if data == '2025-08-21': flash_goes19_dia = flash_goes19_dia21
    if data == '2025-08-22': flash_goes19_dia = flash_goes19_dia22
    if data == '2025-08-23': flash_goes19_dia = flash_goes19_dia23
    if data == '2025-08-24': flash_goes19_dia = flash_goes19_dia24
    if data == 'acumulado':  flash_goes19_dia = flash_goes19_sum

    # quantidade de flashes por dia para a área TOTAL da imagem plotada
    total = int(flash_goes19_dia[:,:].sum(('lon', 'lat')))

    print('Processando dia=', i, ' / ', data, 'raios=', total)

    # figura
    map1 = ax[i].contourf(flash_goes19_dia['lon'],
                          flash_goes19_dia['lat'],
                          flash_goes19_dia[:,:]/64.,
                          cmap='jet',
                          vmin=0.1, vmax=10,
                          levels=uplt.arange(0.1, 10, 1),
                          extend='max')

    # título de cada figura
    if i == 0: ax[i].format(title=data, labels=[True, False, False, False])
    if i == 1: ax[i].format(title=data, labels=[False, False, False, False])
    if i == 2: ax[i].format(title=data, labels=[False, False, False, False])
    if i == 3: ax[i].format(title=data, labels=[True, False, True, False])
    if i == 4: ax[i].format(title=data, labels=[False, False, True, False])
    if i == 5: ax[i].format(title='Acumulado Total', labels=[False, False, True, False])

    # valor de chuva
    ax[i].text(lonmax-11, latmin+0.5, f'Total Relâmpagos: {int(total)}', color='black', fontsize=20)

    # contornos dos ESTADOS
    ax[i].add_geometries(shp_ES, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.6)
    ax[i].add_geometries(shp_RJ, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.6)
    ax[i].add_geometries(shp_SP, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.6)
    ax[i].add_geometries(shp_PR, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.6)
    ax[i].add_geometries(shp_SC, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.6)
    ax[i].add_geometries(shp_RS, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.6)

# barra de cores
fig.colorbar(map1, loc='b', label='relâmpagos/$km^{2}$*dia', ticks=1, ticklabelsize=30, labelsize=30, space=3.1, length=0.55, width=0.6)

# salva figura
fig.save(f'{dir}/output/relampagos_goes19/relampagos_goes19_2025-08-20a24_RS.jpg', dpi=300)

# exibe a figura na tela
uplt.show()

# Contabiliza os relâmpagos por Estado do Brasil.
Esta etapa demora 3min.

In [ ]:
# lendo shapefile dos estados
sp = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/UFs/SP/SP_UF_2019.shp')
pr = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/UFs/PR/PR_UF_2019.shp')
rs = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/UFs/RS/RS_UF_2019.shp')

In [ ]:
# estraindo os flashes de cada estado
flash_sp = flash_goes19_sum.salem.roi(shape=sp).sum(('lat', 'lon'))
flash_pr = flash_goes19_sum.salem.roi(shape=pr).sum(('lat', 'lon'))
flash_rs = flash_goes19_sum.salem.roi(shape=rs).sum(('lat', 'lon'))

In [ ]:
# imprime na tela a quantidade de relâmpagos de cada estado
print('Total de relâmpago em SP', int(flash_sp))
print('Total de relâmpago no PR', int(flash_pr))
print('Total de relâmpago no RS', int(flash_rs))